In [ ]:
import os
from glob import glob

import librosa.display
import numpy as np
from tqdm import tqdm

from audio_augmentations import change_pitch, change_speed, change_value, change_noise
from spectrogram_utils import normalize_signal_length, generate_spectrogram


### Get images

In [ ]:
images = glob("/home/david/expasoft/fart_detector/datasets/eval_mobile/*/*")
print("Total images to process:", len(images))

### Fucntion to process audio and get spectrogram

In [ ]:
def get_spectrogram(
    signal: np.ndarray, 
    sample_rate: int = 16000,
    n_fft: int = 1024,
    hop_length: int = 128,
    sample_length_sec: int = 2,
    use_augmentations=True,
) -> np.ndarray:
    if use_augmentations:
        if np.random.uniform() < 0.2:
            signal = change_pitch(signal, sample_rate)
        if np.random.uniform() < 0.2:
            signal = change_speed(signal)
        if np.random.uniform() < 0.2:
            signal = change_value(signal)
        if np.random.uniform() < 0.2:
            signal = change_noise(signal)

    signal = normalize_signal_length(
        source_signal=signal,
        target_length_sec=sample_length_sec,
        sample_rate=sample_rate,
    )
    signal = (signal - signal.mean()) / signal.std()

    spectrogram = generate_spectrogram(
        signal=signal,
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
    )
    return spectrogram

### Process images

In [ ]:
target_folder = "/home/david/expasoft/fart_detector/datasets/eval_mobile_spectrograms"
augmentation_count = 2

for filename in tqdm(images):
    signal, _ = librosa.load(filename, 16000)

    for num in range(augmentation_count):
        spectr = get_spectrogram(signal)
        
        target_directory = os.path.join(target_folder, filename.split('/')[-2])
        if not os.path.exists(target_directory):
            os.makedirs(target_directory)
            
        target_filename = os.path.join(target_directory, str(num) + '_' + filename.split('/')[-1].split('.')[0] + '.npy')
        
        with open(target_filename, 'wb') as f:
            np.save(f, spectr)